# Case: Previsão do preço de imóveis no Rio de Janeiro utilizando regressão

## Contexto
O mercado imobiliário é um importante setor da economia brasileira, e a previsão do preço de imóveis é uma questão relevante tanto para compradores quanto para vendedores. Neste caso, o objetivo é desenvolver um modelo de regressão para prever o preço de imóveis na cidade do Rio de Janeiro, utilizando dados públicos disponíveis.

## Base de dados
O Inside Airbnb (http://insideairbnb.com/get-the-data.html) oferece dados sobre anúncios de acomodações no Airbnb, incluindo informações sobre localização, tipo de imóvel, número de quartos e preços. É possível utilizar esses dados para modelar o preço dos imóveis no Rio de Janeiro, filtrando as informações relevantes para o contexto local.

## Aplicação de Ciência de Dados

### 1. Coleta e limpeza de dados
Baixe o arquivo "listings.csv" referente à cidade do Rio de Janeiro no Inside Airbnb. Em seguida, realize a limpeza dos dados removendo entradas duplicadas, tratando valores ausentes e convertendo variáveis categóricas em numéricas.
 - bronze: dado coletado em formato legivel (.parquet)
 - silver: dado filtrado ou convertido em formato (string -> date)
 - gold: dado do imóvel (id, preço, quantidade de quartos)

### 2. Análise exploratória de dados (EDA)
A EDA é realizada para identificar padrões, tendências e correlações entre as variáveis. Por exemplo, verificar a distribuição dos preços dos imóveis, identificar quais bairros possuem os imóveis mais caros e analisar a relação entre o tamanho do imóvel e o preço.

### 3. Feature engineering
Crie novas variáveis relevantes, como a distância até o centro da cidade ou a presença de comércio e serviços nas proximidades. Essas variáveis podem ajudar a melhorar o desempenho do modelo de regressão.

### 4. Modelagem e algoritmos
Separe a base de dados em conjuntos de treinamento e teste. Treine diferentes modelos de regressão, como regressão linear, árvores de decisão e regressão de floresta aleatória (Random Forest). Utilize validação cruzada para avaliar o desempenho dos modelos e selecione aquele com a melhor performance.

### 5. Avaliação e interpretação
Avalie o modelo selecionado utilizando métricas de desempenho, como o erro médio quadrático (RMSE) e o coeficiente de determinação (R²). Interprete os resultados, identificando os fatores que mais influenciam o preço dos imóveis e fornecendo insights para os interessados no mercado imobiliário.

Dessa forma, é possível criar um modelo de regressão para prever o preço de imóveis no Rio de Janeiro utilizando dados disponíveis e técnicas de ciência de dados.


#### Carregando os Dados

In [1]:
import pandas as pd
import gzip

In [2]:
mount_path = "/workspaces/prediction_house_price_from_airbnb/Data/Raw/"

In [3]:
listing = pd.read_csv(mount_path + "listings.csv")
neighbourhoods = pd.read_csv(mount_path + "neighbourhoods.csv")
review = pd.read_csv(mount_path + "reviews.csv")


In [6]:
def read_gz(file_name:str)->pd.DataFrame:
    """Le um arquivo gz"""
    with gzip.open(mount_path + file_name, 'rt') as f:
        return pd.read_csv(f)

In [7]:
reviews  = read_gz('reviews.csv.gz')
listings = read_gz('listings.csv.gz')
calendar = read_gz('calendar.csv.gz')

In [9]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 33.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
mount_path = "/workspaces/prediction_house_price_from_airbnb/Data/"

In [13]:
# Salvando as bases
reviews.to_parquet(mount_path + 'bronze/'+'reviews.parquet')
listings.to_parquet(mount_path + 'bronze/'+'listings.parquet')
calendar.to_parquet(mount_path + 'bronze/'+'calendar.parquet')
listing.to_parquet(mount_path + 'bronze/'+'listing.parquet')
neighbourhoods.to_parquet(mount_path + 'bronze/'+'neighbourhoods.parquet')
review.to_parquet(mount_path + 'bronze/'+'review.parquet')
